In [1]:
import pandas as pd
import requests
import numpy as np
import json
from time import sleep
from datetime import datetime
import re

pd.options.display.float_format = '{:.2f}'.format

In [2]:
def lista_cod_ibge():

  url = 'https://raw.githubusercontent.com/aspeddro/siconfipy/main/siconfipy/data/br_cods.csv'

  # Baixar a tabela e criar o DataFrame
  df = pd.read_csv(url)

  df = df[(df['uf']=='SP')&(df['ente']=='São Paulo')]
  # Exibir o DataFrame

  return df

lista_cod_ibge()

,cod_ibge,ente,capital,regiao,uf,esfera,exercicio,populacao,cnpj
3958,3550308,São Paulo,1,SE,SP,M,2021,12325232,46395000000139.00


In [ ]:
def despesa_pessoal():

    df_municipios = lista_cod_ibge()
    todos_rgf = []

    headers = {'accept': 'application/json'}

    for cidade in df_municipios['cod_ibge']:

        for ano in range (2017,2023):

            for quadrimestre in range(1,4):

                esfera = 'M' #ou 'E' para trazer dados estaduais
                poder = 'E' # ou 'L' para trazer dados legislativo
                periodocidade = 'Q' #ou "S" para trazer dados semestrais

                sleep(1.0)

                url = f"http://apidatalake.tesouro.gov.br/ords/siconfi/tt/rgf?an_exercicio={ano}&in_periodicidade={periodocidade}&nr_periodo={quadrimestre}&co_tipo_demonstrativo=RGF&no_anexo=RGF-Anexo%2001&co_esfera={esfera}&co_poder={poder}&id_ente={cidade}"

                try:
                    rgf = requests.get(url).json()
                except:
                    continue

                todos_rgf.append(rgf)

    rgf_anual = pd.concat([pd.DataFrame(bim["items"]) for bim in todos_rgf])

    #filtra para trazer apenas dados da segunda tabela do anexo 1
    rgf_anual = rgf_anual[
            (rgf_anual.loc[:,['coluna']].apply(lambda x:  x.str.contains('Valor|% sobre a RCL Ajustada',na=False, case=False).any(), axis=1))&
            (rgf_anual['anexo']=='RGF-Anexo 01')]


    #retira os hifens e o sinal de (-) indicador de subtração
    rgf_anual['conta'] =  rgf_anual['conta'].apply(lambda x: x.replace(' - ',' ').replace('(-) T', 'T').lower())

    #tira os parentesis e o texto dentro dos parentesis
    #une o texto da coluna conta com o texto da coluna 'coluna'
    rgf_anual['conta_coluna'] = rgf_anual['conta'].apply(lambda x: re.sub("\(.*\)|\s-\s.*", "", x))  + " " + rgf_anual['coluna']

    #deixando as futuras colunas já no padrão e normalizando os campos para evitar que a lei mude e os campos "quebrem"
    rgf_anual['conta_coluna'] = rgf_anual['conta_coluna'].replace({
                  '= receita corrente líquida ajustada  Valor': 'valor_rcl_ajustada',
                 '= receita corrente líquida ajustada para cálculo dos limites da despesa com pessoal  Valor': 'valor_rcl_ajustada',
                 'despesa total com pessoal dtp  Valor': 'valor_dtp',
                 'despesa total com pessoal dtp  % sobre a RCL Ajustada': 'pctg_dtp'})

    #as linhas da coluna conta_coluna passam a ser colunas
    despesa_pessoal = rgf_anual.pivot_table(index=['exercicio','periodo','periodicidade','instituicao','anexo','cod_ibge'], columns='conta_coluna',
                         values='valor', aggfunc='first').sort_values(['exercicio','periodo']).reset_index()

    despesa_pessoal['periodo_numero'] = despesa_pessoal['exercicio'].astype(str) + despesa_pessoal['periodo'].astype(str)

    df_final = despesa_pessoal[['exercicio','periodo','periodo_numero','periodicidade','instituicao','pctg_dtp','valor_dtp','valor_rcl_ajustada']]


    return df_final

df = despesa_pessoal()
df.tail(5)

ValueError: ignored

In [ ]:


df.to_csv("data.csv",index=False)

